In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [22]:

def zs_data(start: str, end: str, src: str) -> pd.DataFrame:
    """Get data from """
    data_df = pd.read_csv(src, index_col=0)#, parse_dates=["Time"])

    freq='15min'
    intervals_per_hour = 4

    # temp bandaid to deal with substation data
    # data_df.index = pd.to_datetime(data_df.index, utc=True)#, format='mixed')

    # NOTE we'll adopt NEM time such that timestamps indicate the end of a 5 min interval
    # Starting with 15 min data we'll set the time index to start at the beninning of this 
    # period and then trim off the first 5 min after ffill()
    tmp = pd.date_range(start=pd.to_datetime("2021-7-1 00:00").tz_localize("UTC"), 
                        end=pd.to_datetime("2023-06-30 23:45").tz_localize("UTC"), 
                        freq=freq)
    data_df.index = tmp

    # data_df = data_df.resample("5min").ffill()#.reset_index()

    # data_df.rename(columns={data_df.columns[0]: "network_load"}, inplace=True)

    new_df = data_df.loc[start:end][1:]

    # new_df.reset_index(inplace=True)

    # temp bandaid to deal with substation data
    # new_df = new_df.rename(columns={data_df.index.name:'settlementdate'})
    # new_df = new_df.rename(columns={'index':'settlementdate'})
    return new_df, intervals_per_hour

In [26]:
zs_df, intervals_per_hour = zs_data("2022-1-1 00:00", "2022-2-1 00:00",
    src = "data/evoenergy-zone-substation-report-2021-23.csv")

# pop_df = pd.read_csv(, index_col=0)#

# http://www.bom.gov.au/climate/averages/tables/cw_070282.shtml
mean_3pm_temps = [26.9,26.4,23.5,19.1,14.9,11.4,10.6,12.6,15.1,18.3,22.0,24.8,18.8]

region = 'Belconnen'

future_frac_heatpumps = 0.75
future_frac_resist = 1 - future_frac_heatpumps

for i_d, day in enumerate(range(1,3)):
    current_day_zs_df = zs_df[region][zs_df.index.dayofyear == day]
    current_day_zs_df.rename('Current', inplace=True)
    # current_day_zs_df.index = current_day_zs_df.index.strftime('%H:%M')
    # current_day_zs_df.index = current_day_zs_df.index.time
    # time_interval = current_day_zs_df.index[1] - current_day_zs_df.index[0]
    # print(current_day_zs_df)
    # print(repr(current_day_zs_df.index))

    current_residences = 10000 #pop_df[region]

    growth_factor = 1.05 # as a growth rate


    month = current_day_zs_df.index[0].month
    temp = mean_3pm_temps[month-1]
    hw_L_per_day = 10.295 - 0.2816*temp
    resist_kWh_per_hw_L = 4
    current_hw_L_per_day = current_residences*hw_L_per_day
    current_hw_MWh_per_day = current_hw_L_per_day*resist_kWh_per_hw_L/1000
    current_hw_MW = current_hw_MWh_per_day/24
        
    curent_day_zs_sans_hw = current_day_zs_df - current_hw_MW
    curent_day_zs_sans_hw.rename('Current w/o hot water', inplace=True)

    future_zs = current_day_zs_df*growth_factor
    future_zs_sans_hw = curent_day_zs_sans_hw*growth_factor

    future_hw_L = growth_factor*current_hw_L_per_day
    future_hw_MWh_resist = future_hw_L*resist_kWh_per_hw_L/1000

    heatpump_CoP = 2.9664 + 0.0703*temp
    forecast_hw_heatpump_MWh = future_frac_heatpumps*future_hw_MWh_resist/heatpump_CoP
    forecast_hw_resist_MWh = future_frac_resist*future_hw_MWh_resist
    future_hw_MWh_total = forecast_hw_heatpump_MWh + forecast_hw_resist_MWh

    timed_hw_start = 9
    timed_hw_end = 16
    time_mask = (current_day_zs_df.index.hour >= timed_hw_start) & \
            (current_day_zs_df.index.hour < timed_hw_end)
    timed_hw_df = pd.Series(index=current_day_zs_df.index, data = 0)
    # timed_hw_df[timed_hw_df.index.hour > timed_hw_start & timed_hw_df.index.hour < timed_hw_end] == 1
    # print(timed_hw_df.between_time(timed_hw_start,timed_hw_end))
    timed_hw_df[time_mask] = 1
    forecast_hw_MW = future_hw_MWh_total / (np.sum(timed_hw_df)/intervals_per_hour)
    timed_hw_df = timed_hw_df*forecast_hw_MW
    # print(timed_hw_df)

    future_zs_with_hw = future_zs_sans_hw + timed_hw_df
    future_zs_with_hw.rename('Future', inplace=True)

    # print(future_zs_with_hw)

    plotting_df = pd.concat([current_day_zs_df, curent_day_zs_sans_hw, future_zs_with_hw],axis=1)

    future_zs_with_hw.index = current_day_zs_df.index.time

    if i_d == 0:
        many_day_df = future_zs_with_hw.rename(day, inplace=True).to_frame()
    else:
        many_day_df = many_day_df.join(future_zs_with_hw.rename(day))
print(many_day_df)


    # future_zs_with_hw.plot()
plotting_df.plot()
    # # print(plotting_df)


                 1         2
00:15:00  15.02207  15.98807
00:30:00  14.75957  15.35807
00:45:00  14.36057  15.05357
01:00:00  14.15057  14.81207
01:15:00  13.86707  14.62307
...            ...       ...
22:45:00  20.09357  20.67107
23:00:00  19.15907  19.74707
23:15:00  18.39257  18.78107
23:30:00  17.57357  17.87807
23:45:00  17.09057  17.33207

[95 rows x 2 columns]
